In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.lines as lines
import IPython.display as ipd
import librosa.display
import numpy as np
from scipy import signal
from scipy.fftpack import fft, fftshift
import dill

## Versuchsbeschreibung:
Im folgenden Versuch werden automatisch zwei Bereiche aus einem Sprachsignals gewählt.
Zu diesen Bereichen werden die Spektren generiert. Entscheiden Sie anhand der Zeitsignale und der zugehörigen Spektren ob es sich jeweils um einen stimmhaften oder einen stimmlosen Bereich handelt. 

In [2]:
# init fig for matplotlib
fig, fig1 = None, None

def handle_select_change(change):
    text.value = showText[label.value]
    
# define file list
wav_list = ['mahlzeit.wav', 'mahlzeitFast.wav', 'mahlzeitLong.wav']

# define select widget
select = widgets.Select(
    options = wav_list,
    value = 'mahlzeit.wav',
    description = 'Dateiliste:')

# define explanations
text_mahlzeit = 'Beschreibung: Das Wort Mahlzeit. Eigenschaften: 16000 Hz, 16 bit mono, Samples: 25333, Länge: 1,58 s'
text_mahlzeit_lang = 'Beschreibung: Das Wort Mahlzeit lang. Eigenschaften: 16000 Hz, 16 bit mono, Samples: 45543, Länge: 2,85 s'
text_mahlzeit_kurz = 'Beschreibung: Das Wort Mahlzeit schnell. Eigenschaften: 16000 Hz, 16 bit mono, Samples: 12580, Länge: 0,79 s'

showText = {"mahlzeit.wav": text_mahlzeit, "mahlzeitFast.wav": text_mahlzeit_kurz, "mahlzeitLong.wav": text_mahlzeit_lang}

label = widgets.Label(value = select.value)
text = widgets.Label(value = showText[label.value])
widgets.link((select, 'value'), (label, 'value'))
select.observe(handle_select_change)

display(select, text)

Select(description='Dateiliste:', options=('mahlzeit.wav', 'mahlzeitFast.wav', 'mahlzeitLong.wav'), value='mah…

Label(value='Beschreibung: Das Wort Mahlzeit. Eigenschaften: 16000 Hz, 16 bit mono, Samples: 25333, Länge: 1,5…

In [3]:
# load audio file
y, sr = librosa.load('wav_files/%s' % select.value, sr = 16000)

startsample=widgets.Text(
    value='0',
    description='Startsample:')

endsample=widgets.Text(
    value=str(y.size),
    description='Endsample:')

display(startsample, endsample)

Text(value='0', description='Startsample:')

Text(value='25333', description='Endsample:')

In [4]:
# close old figure if avaible
if fig:
    plt.close()
    
# define figure
fig, ax = plt.subplots(1, figsize = (9, 7))
ax.axhline(y = 0, color = 'k')
    
y = y[int(startsample.value):int(endsample.value)]
x = np.linspace(0, y.size, num = y.size)

# plot audio
ax.plot(x, y, color = 'r', linewidth = 0.75)
ax.set_xlim(xmin = 0.0, xmax = x[-1])
plt.tight_layout()
# load audio playback module
ipd.Audio(y, rate = sr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
text_startblue = widgets.Text(value = '3000', description = 'Startblue:')
text_endblue = widgets.Text(value = '3500', description = 'Endblue:')
text_startgreen = widgets.Text(value = '10950', description = 'Startgreen:')
text_endgreen = widgets.Text(value = '11470', description = 'Endgreen:')

box = widgets.HBox([widgets.VBox([text_startblue, text_endblue]), widgets.VBox([text_startgreen, text_endgreen])])

display(box)

In [6]:
# fft
def calculate_fft(y_input, fs):
    n = y_input.size
    t = 1 / fs
    y_fft = fft(y_input) / n
    xf = np.linspace(-fs / 2, fs / 2 - fs / n, num = n)
    yf = np.abs(fftshift(y_fft))
    return xf, yf

# close old figures if avaible
if fig:
    plt.close(fig)
    plt.close(fig1)

fig, ax = plt.subplots(1, figsize = (9, 7))
ax.plot(x, y, linewidth=1, color='r')
ax.set_xlim(xmin = x[0], xmax = x[-1])
ax.axhline(y = 0, color = 'k')
ax.grid()
#ax.set_ylim(-0.3, 0.3)

l1 = ax.axvline(x = int(text_startblue.value), color = 'b')
l2 = ax.axvline(x = int(text_endblue.value), color = 'b')
l3 = ax.axvline(x = int(text_startgreen.value), color = 'g')
l4 = ax.axvline(x = int(text_endgreen.value), color = 'g')

fig.tight_layout()

ipd.Audio(y, rate=sr)

fig1, axes = plt.subplots(nrows=2, ncols=2, figsize=(9, 7))

axes[0, 0].set_title("Zeitverlauf blaues Fenster")
axes[0, 0].plot(np.arange(int(text_startblue.value), int(text_endblue.value)), y[int(text_startblue.value):int(text_endblue.value)], color='b')
axes[0, 0].set_xlabel("Sample")
axes[0, 0].set_ylabel("Amplitude")
axes[0, 0].set_xlim(int(text_startblue.value), int(text_endblue.value))
axes[0, 0].grid()

axes[0, 1].set_title("Zeitverlauf grünes Fenster")
axes[0, 1].plot(np.arange(int(text_startgreen.value), int(text_endgreen.value)), y[int(text_startgreen.value):int(text_endgreen.value)], color='g')
axes[0, 1].set_xlabel("Sample")
axes[0, 1].set_ylabel("Amplitude")
axes[0, 1].set_xlim(int(text_startgreen.value), int(text_endgreen.value))
axes[0, 1].grid()

axes[1, 0].set_title("Frequenzspektrum blaues Fenster")
xf_b, yf_b = calculate_fft(y[int(text_startblue.value):int(text_endblue.value)], sr)
axes[1, 0].plot(xf_b, yf_b, color='b')
axes[1, 0].set_xlabel("Frequenz")
axes[1, 0].set_ylabel("Amplitude")
axes[1, 0].set_ylim(ymin = 0)
axes[1, 0].set_xlim(- sr / 2, sr / 2)
axes[1, 0].grid()

axes[1, 1].set_title("Frequenzspektrum grünes Fenster")
xf_g, yf_g = calculate_fft(y[int(text_startgreen.value):int(text_endgreen.value)], sr)
axes[1, 1].plot(xf_g, yf_g, color='g')
axes[1, 1].set_xlabel("Frequenz")
axes[1, 1].set_ylabel("Amplitude")
axes[1, 1].set_ylim(ymin = 0)
axes[1, 1].set_xlim(- sr / 2, sr / 2)
axes[1, 1].grid()

fig1.tight_layout()


save_bg = 'save/radio_select_blue_green.dill'

radio_button_blue = np.array([False, False])
radio_button_green = np.array([False, False])

# try to load checkbox array
try:
    with open(save_bg, 'rb') as fp:
        load_list = dill.load(fp)
        radio_button_blue, radio_button_green = load_list[0], load_list[1]
except:
    pass

radio_button_list_blue = ['stimmlos', 'stimmhaft']
radio_button_list_green = ['stimmlos', 'stimmhaft']

value_radio_blue = [None if len(np.where(radio_button_blue == True)[0]) == 0 else 
               radio_button_list_blue[np.where(radio_button_blue == True)[0][0]]]
value_radio_green = [None if len(np.where(radio_button_green == True)[0]) == 0 else 
               radio_button_list_green[np.where(radio_button_green == True)[0][0]]]

radio_dict = {'blue': widgets.RadioButtons(options=['stimmlos', 'stimmhaft'], 
                                  description='Stimmhaft/-los blaues Fenster', 
                                  style={'description_width': 'initial'}, value=value_radio_blue[0]),
             'green': widgets.RadioButtons(options=['stimmlos', 'stimmhaft'], 
                                   description='Stimmhaft/-los grünes Fenster', 
                                   style={'description_width': 'initial'}, value=value_radio_green[0])}

def callback_checkbox_bg(change):
    # TODO quite dirty...
    if change['type'] == 'change' and change['name'] == 'value':
        if change['owner'].description == 'Stimmhaft/-los blaues Fenster':
            radio_button_blue[radio_button_list_blue.index(change['new'])]\
            = not radio_button_blue[radio_button_list_blue.index(change['new'])]
            if change['old']:
                radio_button_blue[radio_button_list_blue.index(change['old'])]\
                = not radio_button_blue[radio_button_list_blue.index(change['old'])]
        elif change['owner'].description == 'Stimmhaft/-los grünes Fenster':
            radio_button_green[radio_button_list_green.index(change['new'])]\
            = not radio_button_green[radio_button_list_green.index(change['new'])]
            if change['old']:
                radio_button_green[radio_button_list_green.index(change['old'])]\
                = not radio_button_green[radio_button_list_green.index(change['old'])]
                
        # save when changed
        with open(save_bg, 'wb') as fp:
            dill.dump([radio_button_blue, radio_button_green], fp)


items = [ele for ele in radio_dict.values()]
radio_dict['blue'].observe(callback_checkbox_bg)
radio_dict['green'].observe(callback_checkbox_bg)
box_layout = widgets.Layout(display='flex',
                    flex_flow='wrap',
                    position='relative',
                    left='60px',
                    align_items='stretch',
                    width='80%',
                    justify_content='space-between')
box = widgets.Box(children=items, layout=box_layout)
display(box)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Box(children=(RadioButtons(description='Stimmhaft/-los blaues Fenster', index=1, options=('stimmlos', 'stimmha…

In [7]:
# Autograding answer, please ignore


In [8]:
# Autograding answer, please ignore


## 1. Bei stimmhaften Lauten können die Formanten aus dem Frequenzspektrum bestimmt werden.
Das wievielte Maximum der Einhüllenden des Spektrums entspricht dem 1. Formanten?

In [10]:
size_file_max1 = 'save/size_max1.dill'

size_max1 = ''
# try to load checkbox array
try: 
    with open(size_file_max1, 'rb') as fp:
        size_max1 = dill.load(fp)
        if size_max1 == '':
            size_max1 = '0'
except:
    pass

def callback_size_file_max1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        size_max1 = change['new']
        # save when changed
        with open(size_file_max1, 'wb') as fp:
            dill.dump(change['new'], fp)

text_1 = widgets.Text(value=size_max1, continuous_update=True)
bits_label = widgets.Label(value = 'Maximum')
text_1.observe(callback_size_file_max1)

display(widgets.Box([text_1, bits_label]))

Box(children=(Text(value='2'), Label(value='Maximum')))

In [11]:
# Autograding answer, please ignore


## 2. Das wievielte Maximum der Einhüllenden des Spektrums entspricht der Grundfrequenz der Anregungsfunktion?

In [13]:
size_file_max2 = 'save/size_max2.dill'

size_max2 = ''
# try to load checkbox array
try: 
    with open(size_file_max2, 'rb') as fp:
        size_max2 = dill.load(fp)
        if size_max2 == '':
            size_max2 = '0'
except:
    pass

def callback_size_file_max2(change):
    if change['type'] == 'change' and change['name'] == 'value':
        size_max2 = change['new']
        # save when changed
        with open(size_file_max2, 'wb') as fp:
            dill.dump(change['new'], fp)

text_2 = widgets.Text(value=size_max2, continuous_update=True)
bits_label = widgets.Label(value = 'Maximum')
text_2.observe(callback_size_file_max2)

display(widgets.Box([text_2, bits_label]))

Box(children=(Text(value='1'), Label(value='Maximum')))

In [14]:
# Autograding answer, please ignore


## 3. Welche Grundfrequenz hat demnach das "a" in der Sprachdatei mahlzeit.wav?

In [16]:
save_radio_select = 'save/radio_select_base_freqa.dill'

radio_button_array = np.array([False, False, False, False])
# try to load checkbox array
try: 
    with open(save_radio_select, 'rb') as fp:
        radio_button_array = dill.load(fp)
except:
    pass

radio_button_list = ['140 Hz', '400 Hz', '600 Hz', '1000 Hz']
value_radio = [None if len(np.where(radio_button_array == True)[0]) == 0 else 
               radio_button_list[np.where(radio_button_array == True)[0][0]]]

def callback_checkbox(change):
    if change['type'] == 'change' and change['name'] == 'value':
        radio_button_array[radio_button_list.index(change['new'])] = not radio_button_array[radio_button_list.index(change['new'])]
        if change['old']:
            radio_button_array[radio_button_list.index(change['old'])] = not radio_button_array[radio_button_list.index(change['old'])]
        # save when changed
        with open(save_radio_select, 'wb') as fp:
            dill.dump(radio_button_array, fp)

radio_buttons = widgets.RadioButtons(options=radio_button_list, 
                                     value=value_radio[0])

radio_buttons.observe(callback_checkbox)

display(radio_buttons)

RadioButtons(options=('140 Hz', '400 Hz', '600 Hz', '1000 Hz'), value='140 Hz')

In [17]:
# Autograding answer, please ignore
